In [ ]:
__author__ = 'Alice Jacques <alice.jacques@noirlab.edu>, Steve Pothier <steve.pothier@noirlab.edu>, SPARCL team <datalab-spectro@noirlab.edu>'
__version__ = '20240717' # yyyymmdd; 
__datasets__ = ['sdss_dr16', 'boss_dr16', 'desi_edr', 'sdss_dr17_test']

# How to use SPARCL's authentication+authorization feature
Authors: Alice Jacques, Steve Pothier, and the SPARCL team

### Table of contents
* [(1) Login, logout, and check authorized datasets](#loginout)
    - [(1a) A user authorized to access a private dataset](#1a)
    - [(1b) A user unauthorized to access a private dataset](#1b)
    - [(1c) A user that has an SSO account, but is not registered in our Admin](#1c)
    - [(1d) An anonymous user](#1d)
<html><p></html>
* [(2) Auth with the <tt>client.find()</tt> method](#find)
    - [(2a) A user authorized to access a private dataset](#2a)
    - [(2b) A user unauthorized to access a private dataset](#2b)
    - [(2c) A user that has an SSO account, but is not registered in our Admin](#2c)
    - [(2d) An anonymous user](#2d)
<html><p></html>
* [(3) Auth with the <tt>client.retrieve()</tt> method](#retrieve)
    - [(3a) A user authorized to access a private dataset](#3a)
    - [(3b) A user unauthorized to access a private dataset](#3b)
    - [(3c) A user that has an SSO account, but is not registered in our Admin](#3c)
    - [(3d) An anonymous user](#3d)

## NOTE
This notebook demonstrates how users will interact with the Auth feature in SPARCL. It is meant for internal-use only. Connection to sparc1 and sparclstage servers require VPN.

A logged-in session will only last about 30 seconds. Re-run the `client.login` cell to sign back into the specified user account.

# Install the most recent version of the SPARCL client

In [12]:
!pip install sparclclient==1.2.2b11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 3.4 MB/s eta 0:00:00
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: sparclclient
    Found existing installation: sparclclient 1.2.2b10
    Uninstalling sparclclient-1.2.2b10:
      Successfully uninstalled sparclclient-1.2.2b10

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


# Imports and setup

In [1]:
from sparcl.client import SparclClient
from getpass import getpass

/Users/alice.jacques/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#client = SparclClient(url='https://sparc1.datalab.noirlab.edu')  ## PAT
#client = SparclClient(url='http://sparcdev2.csdc.noirlab.edu:8050')  ## DEV
client = SparclClient(url='https://sparclstage.datalab.noirlab.edu')  ## STAGE
#client = SparclClient()  ## PROD
client

(sparclclient:1.2.2b11, api:12.0, https://sparclstage.datalab.noirlab.edu/sparc, client_hash=, verbose=False, connect_timeout=1.1, read_timeout=5400.0)

In [3]:
auth_user = 'test_user_1@noirlab.edu'
unauth_user = 'test_user_2@noirlab.edu'
non_user = 'test_user_3@noirlab.edu'
usrpw = getpass()

 ········


#### Helper function

In [4]:
def count_records(res):
    
    count_sdss=0
    count_boss=0
    count_desi=0
    count_priv=0
    
    for r in res.records:
        if r.data_release == 'SDSS-DR16':
            count_sdss += 1
        elif r.data_release == 'BOSS-DR16':
            count_boss += 1
        elif r.data_release == 'DESI-EDR':
            count_desi += 1
        elif r.data_release == 'SDSS-DR17-test':
            count_priv += 1

    print(f"# of SDSS-DR16 records = {count_sdss}\n"
          f"# of BOSS-DR16 records = {count_boss}\n"
          f"# of DESI-EDR records = {count_desi}\n"
          f"# of SDSS-DR17-test records = {count_priv}")

<a class="anchor" id="loginout"></a>
# (1) Login, logout, and check authorized datasets

In [5]:
client.login?

Signature: client.login(email, password=None)
Docstring:
Login to the SPARCL service.

Args:
    email (:obj:`str`): User login email.

    password (:obj:`str`, optional): User SSO password.
        If not given, the output will prompt the user
        to enter in their SSO password.

Returns:
    None.

Example:
    >>>
    >> client = SparclClient()
    >> client.login('test_user@noirlab.edu', 'testpw')
    Logged in successfully with email='test_user@noirlab.edu'
File:      ~/anaconda3/lib/python3.8/site-packages/sparcl/client.py
Type:      method

In [6]:
client.logout?

Signature: client.logout()
Docstring:
Logout of the SPARCL service.

Args:
    None.

Returns:
    None.

Example:
    >>> client = SparclClient()
    >>> client.logout()
    Logged-out successfully.  Previously logged-in with email None.
File:      ~/anaconda3/lib/python3.8/site-packages/sparcl/client.py
Type:      method

<a class="anchor" id="1a"></a>
### (1a) A user authorized to access a private dataset

In [7]:
client.login(auth_user, usrpw)

Logged in successfully with email='test_user_1@noirlab.edu'


In [8]:
client.authorized

{'Loggedin_As': 'test_user_1@noirlab.edu',
 'Authorized_Datasets': {'BOSS-DR16',
  'DESI-EDR',
  'SDSS-DR16',
  'SDSS-DR17-test'}}

In [9]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_1@noirlab.edu.


<a class="anchor" id="1b"></a>
### (1b) A user unauthorized to access a private dataset

In [10]:
client.login(unauth_user, usrpw)

Logged in successfully with email='test_user_2@noirlab.edu'


In [11]:
client.authorized

{'Loggedin_As': 'test_user_2@noirlab.edu',
 'Authorized_Datasets': {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}}

In [12]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_2@noirlab.edu.


<a class="anchor" id="1c"></a>
### (1c) A user that has an SSO account, but is not registered in our Admin

In [13]:
client.login(non_user, usrpw)

Logged in successfully with email='test_user_3@noirlab.edu'


In [14]:
client.authorized

{'Loggedin_As': 'Anonymous',
 'Authorized_Datasets': {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}}

In [15]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_3@noirlab.edu.


<a class="anchor" id="1d"></a>
### (1d) An anonymous user

In [16]:
client.authorized

{'Loggedin_As': 'Anonymous',
 'Authorized_Datasets': {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}}

In [17]:
client.logout()

Logged-out successfully.  Previously logged-in with email None.


<a class="anchor" id="find"></a>
# (2) Auth with the `client.find()` method

In [24]:
# No dataset(s) specified in constraints
cons = {'spectype': ['GALAXY'],
        'redshift': [0.299649, 0.299652]}
# Private dataset specified in constraints
cons2 = {'spectype': ['GALAXY'],
         'redshift': [0.299649, 0.299652],
         'data_release': ['SDSS-DR17-test', 'DESI-EDR']}
# Outfields
out = ['sparcl_id', 'data_release']

<a class="anchor" id="2a"></a>
### (2a) A user authorized to access a private dataset

In [26]:
client.login(auth_user, usrpw)

Logged in successfully with email='test_user_1@noirlab.edu'


In [27]:
# No dataset(s) specified in constraints.
auth_found = client.find(outfields=out,
                         constraints=cons,
                         limit=20000)
count_records(auth_found)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 1


In [28]:
# Private dataset specified in constraints.
auth_found2 = client.find(outfields=out,
                          constraints=cons2,
                          limit=20000)
count_records(auth_found2)

# of SDSS-DR16 records = 0
# of BOSS-DR16 records = 0
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 1


In [22]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_1@noirlab.edu.


<a class="anchor" id="2b"></a>
### (2b) A user unauthorized to access a private dataset

In [29]:
client.login(unauth_user, usrpw)

Logged in successfully with email='test_user_2@noirlab.edu'


In [30]:
# No dataset(s) specified in constraints.
unauth_found = client.find(outfields=out,
                           constraints=cons,
                           limit=20000)
count_records(unauth_found)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


In [31]:
# Private dataset specified in constraints.
# This cell should produce an "Access Not Allowed" error.
unauth_found2 = client.find(outfields=out,
                            constraints=cons2,
                            limit=20000)
count_records(unauth_found2)

AccessNotAllowed: [DSDENIED] uname='test_user_2@noirlab.edu' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

In [32]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_2@noirlab.edu.


<a class="anchor" id="2c"></a>
### (2c) A user that has an SSO account, but is not registered in our Admin

In [33]:
client.login(non_user, usrpw)

Logged in successfully with email='test_user_3@noirlab.edu'


In [34]:
# No dataset(s) specified in constraints.
non_found = client.find(outfields=out,
                        constraints=cons,
                        limit=20000)
count_records(non_found)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


In [35]:
# Private dataset specified in constraints.
# This cell should produce an "Access Not Allowed" error.
non_found2 = client.find(outfields=out,
                         constraints=cons2,
                         limit=20000)
count_records(non_found2)

AccessNotAllowed: [DSDENIED] uname='ANONYMOUS' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

In [36]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_3@noirlab.edu.


<a class="anchor" id="2d"></a>
### (2d) An anonymous user

In [37]:
# No dataset(s) specified in constraints.
anon_found = client.find(outfields=out,
                         constraints=cons,
                         limit=20000)
count_records(anon_found)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


In [38]:
# Private dataset specified in constraints.
# This cell should produce an "Access Not Allowed" error.
anon_found2 = client.find(outfields=out,
                          constraints=cons2,
                          limit=20000)
count_records(anon_found2)

AccessNotAllowed: [DSDENIED] uname='ANONYMOUS' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

<a class="anchor" id="retrieve"></a>
# (3) Auth with the `client.retrieve()` method

<a class="anchor" id="3a"></a>
### (3a) A user authorized to access a private dataset

In [39]:
client.login(auth_user, usrpw)

Logged in successfully with email='test_user_1@noirlab.edu'


In [40]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
auth_ret = client.retrieve(uuid_list=auth_found.ids,
                           include=out,
                           limit=20000)
count_records(auth_ret)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 1


In [41]:
# uuid_list includes IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
auth_ret2 = client.retrieve(uuid_list=auth_found2.ids,
                            include=out,
                            dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                            limit=20000)
count_records(auth_ret2)

# of SDSS-DR16 records = 0
# of BOSS-DR16 records = 0
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 1


In [42]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_1@noirlab.edu.


<a class="anchor" id="3b"></a>
### (3b) A user unauthorized to access a private dataset

In [43]:
client.login(unauth_user, usrpw)

Logged in successfully with email='test_user_2@noirlab.edu'


In [44]:
# uuid_list does not include IDs from private dataset, no dataset(s) specified.
unauth_ret = client.retrieve(uuid_list=unauth_found.ids,
                             include=out,
                             limit=20000)
count_records(unauth_ret)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


In [45]:
# uuid_list does not include IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
unauth_ret2 = client.retrieve(uuid_list=unauth_found.ids,
                              include=out,
                              dataset_list=['SDSS-DR17-test','DESI-EDR'],
                              limit=20000)
count_records(unauth_ret2)

AccessNotAllowed: [DSDENIED] uname='test_user_2@noirlab.edu' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

In [46]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
# This cell should only retrieve records from user-authorized datasets
# and output a UserWarning: Some UUIDs were not found.
unauth_ret3 = client.retrieve(uuid_list=auth_found.ids,
                              include=out,
                              limit=20000)
count_records(unauth_ret3)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


/var/folders/6j/v2lh3ssj13g6fs_lhk8nlps40000gp/T/ipykernel_72333/1687955117.py:4: UserWarning: Some UUIDs were not found. 1 out of the 15 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 1 unavailable uuids.
  unauth_ret3 = client.retrieve(uuid_list=auth_found.ids,


In [47]:
# uuid_list includes IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
unauth_ret4 = client.retrieve(uuid_list=auth_found.ids,
                              include=out,
                              dataset_list=['SDSS-DR17-test','DESI-EDR'],
                              limit=20000)
count_records(unauth_ret4)

AccessNotAllowed: [DSDENIED] uname='test_user_2@noirlab.edu' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

In [48]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_2@noirlab.edu.


<a class="anchor" id="3c"></a>
### (3c) A user that has an SSO account, but is not registered in our Admin

In [49]:
client.login(non_user, usrpw)

Logged in successfully with email='test_user_3@noirlab.edu'


In [50]:
# uuid_list does not include IDs from private dataset, no dataset(s) specified.
non_ret = client.retrieve(uuid_list=non_found.ids,
                          include=out,
                          limit=20000)
count_records(non_ret)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


In [51]:
# uuid_list does not include IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
non_ret2 = client.retrieve(uuid_list=non_found.ids,
                           include=out,
                           dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                           limit=20000)
count_records(non_ret2)

AccessNotAllowed: [DSDENIED] uname='ANONYMOUS' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

In [52]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
# This cell should only retrieve records from user-authorized datasets
# and output a UserWarning: Some UUIDs were not found.
non_ret3 = client.retrieve(uuid_list=auth_found.ids,
                           include=out,
                           limit=20000)
count_records(non_ret3)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


/var/folders/6j/v2lh3ssj13g6fs_lhk8nlps40000gp/T/ipykernel_72333/2556312795.py:4: UserWarning: Some UUIDs were not found. 1 out of the 15 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 1 unavailable uuids.
  non_ret3 = client.retrieve(uuid_list=auth_found.ids,


In [53]:
# uuid_list includes IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
non_ret4 = client.retrieve(uuid_list=auth_found.ids,
                           include=out,
                           dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                           limit=20000)
count_records(non_ret4)

AccessNotAllowed: [DSDENIED] uname='ANONYMOUS' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

In [54]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_3@noirlab.edu.


<a class="anchor" id="3d"></a>
### (3d) An anonymous user

In [55]:
# uuid_list does not include IDs from private dataset, no dataset(s) specified.
anon_ret = client.retrieve(uuid_list=anon_found.ids,
                           include=out,
                           limit=20000)
count_records(anon_ret)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


In [56]:
# uuid_list does not include IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
anon_ret2 = client.retrieve(uuid_list=anon_found.ids,
                            include=out,
                            dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                            limit=20000)
count_records(anon_ret2)

AccessNotAllowed: [DSDENIED] uname='ANONYMOUS' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']

In [57]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
# This cell should only retrieve records from user-authorized datasets
# and output a UserWarning: Some UUIDs were not found.
anon_ret3 = client.retrieve(uuid_list=auth_found.ids,
                            include=out,
                            limit=20000)
count_records(anon_ret3)

# of SDSS-DR16 records = 2
# of BOSS-DR16 records = 9
# of DESI-EDR records = 3
# of SDSS-DR17-test records = 0


/var/folders/6j/v2lh3ssj13g6fs_lhk8nlps40000gp/T/ipykernel_72333/134315787.py:4: UserWarning: Some UUIDs were not found. 1 out of the 15 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 1 unavailable uuids.
  anon_ret3 = client.retrieve(uuid_list=auth_found.ids,


In [58]:
# uuid_list includes IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
anon_ret4 = client.retrieve(uuid_list=auth_found.ids,
                            include=out,
                            dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                            limit=20000)
count_records(anon_ret4)

AccessNotAllowed: [DSDENIED] uname='ANONYMOUS' is declined access to datasets=['SDSS-DR17-test']; drs_requested=['DESI-EDR', 'SDSS-DR17-test'] my_auth=['BOSS-DR16', 'DESI-EDR', 'SDSS-DR16']